In [1]:
import os, sys
print(sys.path)

['C:\\Users\\joelj\\AppData\\Local\\Programs\\Python\\Python313\\python313.zip', 'C:\\Users\\joelj\\AppData\\Local\\Programs\\Python\\Python313\\DLLs', 'C:\\Users\\joelj\\AppData\\Local\\Programs\\Python\\Python313\\Lib', 'C:\\Users\\joelj\\AppData\\Local\\Programs\\Python\\Python313', '', 'C:\\Users\\joelj\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages']


In [3]:
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'numpy'

In [ ]:
df_0 = pd.read_csv('D:/Projects/PowerBI/Vehicle_prices_aus/Aus_vehicles_prices_cleaned_1.csv')

In [ ]:
df_0

In [ ]:
new_names = {'Car/Suv':'segment','Brand': 'brand', 'Year': 'year', 'Model':'model', 'Vehicle_type':'vehicle_type', 'Title':'title', 'UsedOrNew':'used_or_new', 'Transmission':'transmission', 'Engine':'engine', 'DriveType':'drive_type', 'FuelType':'fuel_type', 'FuelConsumption':'fuel_consumption', 'Kilometres':'kilometres', 'ColourExtInt':'color_ext_int', 'Location': 'location', 'CylindersinEngine': 'cylinders','BodyType':'body_type', 'Doors':'doors','Seats':'seats','Price': 'price'}
df = df_0.rename(columns=new_names)

In [ ]:
df.columns

## Analysing Used or new

In [ ]:
new_cars = df.loc[df['used_or_new']=='NEW','year']
new_cars.groupby(new_cars.values).count()
# df['used_or_new'].groupby(df['used_or_new'].values).count()

## Column brand

In [ ]:
df['brand'].unique()


In [ ]:
type(df['brand'])

## Column Year

In [ ]:
df['year'].unique()

## Column model

In [ ]:
df['model'].unique()

In [ ]:
df['model'].isna().sum()

## Column segment

In [ ]:
df['segment'].unique()

In [ ]:
segment_count = df['segment'].groupby(df['segment'].values).count()
segment_count

In [ ]:
df['segment'].values

In [ ]:
segment_count.sort_values(ascending=False)[:15]

In [ ]:
# dropping the segmwnt column because its not very useful
df = df.drop(labels = 'segment', axis = 1)
df

## Column title

In [ ]:
df['title']

In [ ]:
df = df.drop( labels = 'title', axis=1 )
df

## Column used_or_new

In [ ]:
df.columns

In [ ]:
df['used_or_new']

In [ ]:
df['used_or_new'].unique()

## Column transmission

In [ ]:
df['transmission'].unique()

In [ ]:
df.loc[df['transmission']== '-']

In [ ]:
df['transmission'] = df['transmission'].replace(to_replace='-' ,value=np.nan)

In [ ]:
df.loc[df['transmission']== '-']

In [ ]:
type(df.loc[df['transmission'].isna()]['transmission'][6340]) # checking for correct change to nan

## Column engine

In [ ]:
df['engine']

In [ ]:
# # Checking if all values in the col are of the regex of num of cylinders and capacity in litres
df['engine'].str.fullmatch(r'\d{1,2} cyl, \d+(?:\.\d+)? L').all()

In [ ]:
# What are the values that dont conform to the pattern
df.loc[ ~df['engine'].str.fullmatch(r'\d{1,2} cyl, \d+(?:\.\d+)? L')]['engine'].unique()

In [ ]:
# 1763 messy rows in engine column
df.loc[ ~df['engine'].str.fullmatch(r'\d{1,2} cyl, \d+(?:\.\d+)? L')]['engine'].count()

In [ ]:
# Which entries have engine 0l but non electric engine
df.loc[(df['engine'] == '0 L') & ~(df['fuel_type'] == 'Electric')]

# Google search shows NISSAN LEAF is an electric car. 

In [ ]:
# Change entry 7936 engine value to NaN
df.loc[7936, 'engine'] = np.nan
df['engine'][7936]

In [ ]:
# Changing all -'s to NaN'
df.loc[(df['engine'] == '-')] = np.nan

In [ ]:
# Entry 6637 
df.loc[df['engine'] =='4 cyl']

In [ ]:
# ENTRY  10029 
df.loc[df['engine'] =='2 L']

## Column cylinder_capacity

In [ ]:
cyl_capacity = pd.Series()

In [ ]:
cyl_capacity = df['engine'].str.strip().str.extract(r'[\s,]+(\d*\.?\d+)\s*L\b', expand=False).astype(float)

In [ ]:
cyl_capacity.isna().sum() # We have 1763 NaN values consistent with the count we found not following the regex patterm

In [ ]:
df['cylinder_capacity'] = cyl_capacity

In [ ]:
df.drop(labels='engine', axis=1,inplace=True)

## Column drive_type

In [ ]:
df['drive_type'].unique()

In [ ]:
df.loc[df['drive_type'] == 'Other']

In [ ]:
df['drive_type'] = df['drive_type'].replace(to_replace='Other' ,value=np.nan)

In [ ]:
df['drive_type'].unique()

## Column fuel_type

entry #7936 nissan leaf is electric

In [ ]:
df['fuel_type'].unique()

In [ ]:
# Change missing values to nan
df['fuel_type'] = df['fuel_type'].replace(to_replace = '-', value = np.nan)
df['fuel_type'].unique()

## Column fuel_consumption

In [ ]:
df['fuel_consumption']

In [ ]:
# Check sanity
df['fuel_consumption'].str.fullmatch(r'(\d+(?:\.\d+)?)\s*L\s*/\s*100\s*km').all()

In [ ]:
df['fuel_consumption'].str.fullmatch(r'(\d+(?:\.\d+)?)\s*L\s*/\s*100\s*km')

In [ ]:
pattern_conforming_boolean = df['fuel_consumption'].str.fullmatch(r'(\d+(?:\.\d+)?)\s*L\s*/\s*100\s*km').astype('bool')

In [ ]:
df['fuel_consumption'].loc[(~pattern_conforming_boolean)].unique()

In [ ]:
df['fuel_consumption'] = df['fuel_consumption'].replace(to_replace='-', value = np.nan)

In [ ]:
df['fuel_consumption'].loc[(~pattern_conforming_boolean)].unique()

## Column kilometres

In [ ]:
df['kilometres']

In [ ]:
(df['kilometres'] > 0).pipe(all)

## Column color_ext_int

In [ ]:
df['color_ext_int']

In [ ]:
df[['exterior_clr', 'interior_clr']]  = df['color_ext_int'].str.extract(r'^\s*(.*?)\s*\/\s*(.*?)\s*$')

In [ ]:
df['exterior_clr'].unique()

In [ ]:
non_colors = {'-', 'Other', '5 years', '3 years', '7 years', '2 years',
              '1 years', '6 years', '4 years'}

df['exterior_clr'] = df['exterior_clr'].replace( to_replace = non_colors, value = np.nan )

df['exterior_clr'].unique()

In [ ]:
df['interior_clr'].unique()

In [ ]:
df['interior_clr'].groupby(df['interior_clr'].values).count().sort_values(ascending=False)

In [ ]:
df.drop(labels='color_ext_int', axis = 1, inplace=True)

## Column Interior Color

In [ ]:
df['interior_clr'].groupby(df['interior_clr'].values).count().sort_values(ascending=False)[:10]

In [ ]:
df.drop(labels='interior_clr',axis=1, inplace=True )

In [ ]:
df.columns

## Column location

In [ ]:
df['location']

In [ ]:
# Extracts two new columns: suburb and state
df[['suburb', 'state']] = df['location'].str.extract(r'^\s*(.+?),\s*([A-Z]{2,})\s*$')

In [ ]:
df['state']

In [ ]:
df['suburb']

In [ ]:
df.drop(labels='location', axis=1, inplace=True)

## Column Cylinders

In [ ]:
df['cylinders']

In [ ]:
df['cylinders'].unique()

In [ ]:
invalid_values = {'0 L', '2 L'}
df['cylinders'] = df['cylinders'].replace( to_replace = invalid_values, value = np.nan )
df['cylinders'].unique()

In [ ]:
df['cylinders'] = df['cylinders'].str.extract(r'(\d+)')
df['cylinders']

In [ ]:
df['cylinders'].unique()

## Column body_type

In [ ]:
df['body_type'].unique()

In [ ]:
df['body_type'].unique()[-1]

In [ ]:
df['body_type'] = df['body_type'].replace(to_replace = 'Ute / Tray', value='Ute')

In [ ]:
df['body_type'].unique()

## Column Doors

In [ ]:
df['doors'] = df['doors'].str.extract(r'(\d+)')

In [ ]:
df['doors'].unique()

## Column Seats

In [ ]:
df['seats'].unique()

In [ ]:
df['seats'] = df['seats'].str.extract(r'(\d+)')

In [ ]:
df['seats'].unique()

## Column price

In [ ]:
df['price']

In [ ]:
df['price'].isna().sum()

In [ ]:
df

## Serialization of data

In [ ]:
df.to_csv('D:/Projects/PowerBI/Vehicle_prices_aus/aus_vehicle_prices_cleaned.csv')